In [84]:
! /usr/local/Cellar/sumo/1.16.0/share/sumo/tools/route/route2OD.py \
    -r ../network/BOS/trips24h_smoothed.rou.xml \
    -a ../network/BOS/taz.xml \
    -o ../network/BOS/init.xml


read 969 TAZ
edges ['-261123523#3', '-261123523#5', '-426484634#1', '-426492037#1', '-426611253'] (total 20672) are sources for more than one TAZ
edges ['-261123523#3', '-261123523#5', '-426484634#1', '-426492037#1', '-426611253'] (total 20672) are sinks for more than one TAZ
read 3806510 vehicles
Wrote 278738 OD-pairs (278738 distinct) in 1 intervals (3806510 vehicles total)


### Learning how to parse an xlm file

In [76]:
import xml.etree.ElementTree as ET
tree = ET.parse('../network/BOS/out.xml')
root = tree.getroot()


In [77]:
root[0][0].attrib

{'from': '100', 'to': '100', 'count': '492'}

In [78]:
root[0][0].attrib["count"] = "1"

In [79]:
root[0][0].attrib

{'from': '100', 'to': '100', 'count': '1'}

In [82]:
import numpy as np

dim = 10
x = np.random.rand(dim)*2000000

for i,x in enumerate(x):
    root[0][i].attrib["count"] = str(np.round(x))


In [83]:
def generate_od_xml(x, config, sim_setup, prefix_run):

    # check if initial 


    od_xml_str = (
        f"<?xml version=\"1.0\" encoding=\"UTF-8\"?> \n"
        f"<data xmlns:xsi=\"http://www.w3.org/2001/XMLSchema-instance\" xsi:noNamespaceSchemaLocation=\"http://sumo.dlr.de/xsd/datamode_file.xsd\">\n"
        f"<interval id=\"CarA\" begin=\"54000\" end=\"57600.0\">\n"
        f"<tazRelation from=\"taz91\" to=\"taz93\" count=\"{int(np.floor(od_rand[0]))}\"/>\n"
        f"<tazRelation from=\"taz91\" to=\"taz94\" count=\"{int(np.floor(od_rand[1]))}\"/>\n"
        f"<tazRelation from=\"taz92\" to=\"taz93\" count=\"{int(np.floor(od_rand[2]))}\"/>\n"
        f"<tazRelation from=\"taz92\" to=\"taz94\" count=\"{int(np.floor(od_rand[3]))}\"/>\n"
        f"</interval>\n"
        f"</data>"
    )

    file_name = f"{config['NETWORK']}/{prefix_run}_{sim_setup['current_od']}"
    print('printing '+file_name)
    f = open(file_name,'w')
    f.write(od_xml_str) 
    f.close()
